In [ ]:
import skimage
import skimage.io
import skimage.transform

import os
import scipy as scp
import scipy.misc

import numpy as np
import logging
import tensorflow as tf
import sys

import matplotlib.pyplot as plt

logging.basicConfig(format='%(asctime)s %(levelname)s %(message)s',
                    level=logging.INFO,
                    stream=sys.stdout)

img1 = skimage.io.imread("./trees.png")

% matplotlib inline

In [ ]:
plt.figure(1)
plt.cla()
plt.imshow(img1)
print("Click desired cut position")
#click = plt.ginput()
plt.close('all')


In [ ]:
my_dpi = 72
center = (650,358)
radius = 642
zero_pixel = img1[0,0]
img1.shape

In [ ]:
indices = np.array([(i,j) for j in range(img1.shape[1]) for i in range(img1.shape[0])])
ii = np.array([[i for j in range(img1.shape[1])] for i in range(img1.shape[0])])
jj = np.array([[j for j in range(img1.shape[1])] for i in range(img1.shape[0])])

In [ ]:
xx = (jj - center[0]) / radius
yy = (ii - center[1]) / radius
rr = np.sqrt(xx * xx + yy * yy)
sr = np.maximum(np.abs(xx/np.max(xx)), np.abs(yy/np.max(yy)))
sr = rr * sr + (1-rr) * rr
th = np.arctan2(xx, yy)

In [ ]:
plt.figure("circle")
plt.cla()
img2 = img1
img2[rr > 1] = 255
plt.imshow(img2)
plt.axvline(center[0], color='k')
plt.axhline(center[1], color='k')
plt.show()

In [ ]:
plt.contour(xx,yy,th,100,label="angle")
plt.contour(xx,yy,sr,100,label="square radius")
#plt.contour(xx,yy,rr,100,lable="circular radius")
plt.show()

In [ ]:
def polar_to_xy(r, theta):
    x = r * np.sin(theta)
    y = r * np.cos(theta)
    return x,y 
def xy_to_idx(x,y):
    center = (650,358)
    radius = 642
    j = x * radius + center[0]
    i = y * radius + center[1]
    j = np.round(j).astype(int)
    i = np.round(i).astype(int)
    return i,j

In [ ]:
def find_nearest(array,value, array2, value2):
  idx = (np.abs(array-value)+np.abs(array2-value2)).argmin()
  return np.unravel_index(idx, array.shape)

def create_map():
  global img1, ii, indices, sr, rr, th
  global progress, STOP
  global undist, map_
  global xy_to_idx, polar_to_xy
  STOP = False

  progress = 1/len(indices)
  for i,j in indices:
    _, _, idx = single_op((i,j), (rr, sr, th, xy_to_idx, polar_to_xy))
    map_.append(((i, j), idx))
    progress = progress + 1/len(indices)
    if STOP:
        break

def apply_map(img1, map_):
  undist = img1 * 0
  for (i,j), (dist_i, dist_j) in map_:
    if dist_i < 0: dist_i = np.nan
    if dist_j < 0: dist_j = np.nan
    try:
      undist[i,j] = img1[dist_i, dist_j]
    except:
      undist[i,j] = img1[0,0]
  return undist

from functools import partial

def single_op(ij, fixed_data=None):
     rr, sr, th, xy_to_idx, polar_to_xy = fixed_data
     i,j = ij
     r = sr[i,j]
     theta = th[i,j]
     x, y = polar_to_xy(r, theta)
     idx = xy_to_idx(x,y)
     return i,j, idx

partial_single_op = partial(single_op, fixed_data=(rr, sr, th, xy_to_idx, polar_to_xy))

In [ ]:
import threading

map_ = []
undist = img1 * 0
consumable_indices = list(indices)
progress = 0
STOP = False


t = threading.Thread(target=create_map)
t.start()


In [ ]:
print(progress)
a = undist = apply_map(img1, map_)
plt.figure(figsize=(20,10))
plt.imshow(a, cmap='gray')

In [ ]:
STOP = True
del t

In [ ]:
with open('map.csv','w') as f:
    for item in map_:
        f.write(item.__str__())
        f.write('\n')

In [ ]:
dir_ = './distorted/'
dir_out = './undistorted/'
if not os.path.exists(dir_out): os.makedirs(dir_out)
img_names = os.listdir(dir_)
for name in img_names:
   distorted = skimage.io.imread(dir_+name)[:,:,0]
   undistorted = apply_map(distorted, map_)
   skimage.io.imsave(dir_out+name, undistorted)

# Vertical Lines

In [ ]:
from skimage.transform import (hough_line, hough_line_peaks,
                               probabilistic_hough_line)
from skimage.feature import canny

In [ ]:
plt.figure("Lines", figsize=(20,10))
plt.clf()
img = skimage.io.imread(dir_+name)
plt.imshow(img,cmap='gray' )

h, theta, d = hough_line(img)
for _, angle, dist in zip(*hough_line_peaks(h, theta, d)):
    y0 = (dist - 0 * np.cos(angle)) / np.sin(angle)
    y1 = (dist - img.shape[1] * np.cos(angle)) / np.sin(angle)
    plt.plot((0, img.shape[1]), (y0, y1), '-r')
plt.xlim((0, img.shape[1]))
plt.ylim((img.shape[0], 0))
del h, theta, d

In [ ]:
dir_ = './undistorted/'
dir_out = './lines/'
if not os.path.exists(dir_out): os.makedirs(dir_out)
#img_names = sorted(os.listdir(dir_))
img_names = os.listdir(dir_)
for name in img_names:
    img = skimage.io.imread(dir_+name)
    # Canny Edges
    fig = plt.figure("Canny edges", frameon=False, figsize=(img.shape[1]/my_dpi, img.shape[0]/my_dpi), dpi=my_dpi)
    plt.clf()
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    edges = canny(img, 2, 1, 25)
    plt.imshow(edges, cmap='gray')
    # Probabilistic Lines
    fig = plt.figure("Probabilistic Lines",frameon=False,
                     figsize=(img.shape[1]/my_dpi, img.shape[0]/my_dpi), dpi=my_dpi)
    plt.clf()
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    plt.imshow(img, cmap='gray')
    lines = probabilistic_hough_line(edges, threshold=10, line_length=10, line_gap=3)
    vertical_lines = []
    for line in lines:
        p0, p1 = line
        p1 = np.array(p1)
        p0 = np.array(p0)
        vec = p1 - p0
        if np.abs(vec[1]/(vec[0]+1e-1)) > 2:
            vertical_lines.append(line)
    for line in vertical_lines:
        p0, p1 = line
        plt.plot((p0[0], p1[0]), (p0[1], p1[1]), 'red')
    plt.xlim((0, img.shape[1]))
    plt.ylim((img.shape[0], 0))
    plt.savefig(dir_out+name)
    plt.show()